In [1]:
!pip install langchain transformers torch accelerate

In [2]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Enter your own HugginFace acess token here"

In [3]:
!pip install -U langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [12]:
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import yfinance as yf

# Step 1: Define your LLM
model_name = "google/flan-t5-large"
hf_pipeline = pipeline("text2text-generation", model=model_name)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Step 2: Get stock data (last 5 days for example)
ticker = "AMZN"
data = yf.download(ticker, period="5d", interval="1d")
print(data.head())
print(data.columns)
recent_prices = data[("Close", ticker)].to_list()
# Format stock data as context
stock_context = ", ".join([f"{p:.2f}" for p in recent_prices])

print("Recent closing prices:", recent_prices)

# Step 3: Define a prediction prompt
template = """You are a financial assistant.
Here are the last few closing prices for {ticker}: {stock_data}

Based on this trend, what do you predict about {ticker}'s short-term movement (up, down, or stable)?
Also, provide reasoning in simple terms."""

prompt = PromptTemplate(
    template=template,
    input_variables=["ticker", "stock_data"]
)

# Step 4: Create an LLM chain
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Step 5: Run it
result = llm_chain.run({"ticker": ticker, "stock_data": stock_context})
print(result)

Device set to use cpu
/tmp/ipython-input-296172939.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5d", interval="1d")
[*********************100%***********************]  1 of 1 completed


Price            Close        High         Low        Open    Volume
Ticker            AMZN        AMZN        AMZN        AMZN      AMZN
Date                                                                
2025-09-08  235.839996  237.600006  233.750000  234.940002  33947100
2025-09-09  238.240005  238.850006  235.080002  236.360001  27033800
2025-09-10  230.330002  237.679993  229.100006  237.520004  60907700
2025-09-11  229.949997  231.529999  229.339996  231.490005  37427500
2025-09-12  228.039993  230.789993  226.289993  230.304993  27168497
MultiIndex([( 'Close', 'AMZN'),
            (  'High', 'AMZN'),
            (   'Low', 'AMZN'),
            (  'Open', 'AMZN'),
            ('Volume', 'AMZN')],
           names=['Price', 'Ticker'])
Recent closing prices: [235.83999633789062, 238.24000549316406, 230.3300018310547, 229.9499969482422, 228.0399932861328]
The last closing prices for AMZN are: 235.84, 238.24, 230.33, 229.95, 228.04 The short-term movement for AMZN is up. The answer:

In [17]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Step 1: Ask user for stock symbol
ticker = input("Enter stock ticker (e.g., AAPL, TSLA, MSFT): ").upper()

# Step 2: Download 6 months of historical data
data = yf.download(ticker, period="6mo", interval="1d")

# Step 3: Extract 'Close' prices as a Series
# Handle MultiIndex from yfinance
if ("Close", ticker) in data.columns:
    close_series = data[("Close", ticker)]  # Series
else:
    close_series = data["Close"]  # fallback

# Step 4: Reset index and create df for Prophet
df = pd.DataFrame({
    "ds": close_series.index,   # Date
    "y": close_series.values   # Close price as numeric
})

# Step 5: Ensure types
df["ds"] = pd.to_datetime(df["ds"])
df["y"] = pd.to_numeric(df["y"], errors="coerce")

print(df.head())  # Debug check

# Train Prophet
model = Prophet(daily_seasonality=True)
model.fit(df)

# Step 6: Train Prophet model
model = Prophet(daily_seasonality=True)
model.fit(df)

# Step 7: Predict next 7 days
future = model.make_future_dataframe(periods=7)
forecast = model.predict(future)

# Get last 5 forecasted values
predicted = forecast.tail(5)[["ds", "yhat", "yhat_lower", "yhat_upper"]]

# Convert to string for LLM
pred_text = "\n".join(
    f"{row.ds.date()}: Predicted {row.yhat:.2f} (range {row.yhat_lower:.2f} - {row.yhat_upper:.2f})"
    for row in predicted.itertuples()
)

# Step 8: Define LLM (Flan-T5 for explanation)
hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-large")
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Step 9: Prompt for explanation
template = """You are a financial assistant.
Here are Prophet's stock price predictions for {ticker}:

{forecast}

Please summarize the trend (is the price going up, down, or stable) and give a simple explanation.
"""
prompt = PromptTemplate(template=template, input_variables=["ticker", "forecast"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Step 10: Run the chain
result = llm_chain.run({"ticker": ticker, "forecast": pred_text})

print("\n--- Prophet Predictions ---")
print(pred_text)
print("\n--- LLM Summary ---")
print(result)

Enter stock ticker (e.g., AAPL, TSLA, MSFT): AAPL


/tmp/ipython-input-2530338300.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="6mo", interval="1d")
[*********************100%***********************]  1 of 1 completed
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoaddiawr/so7yjbj9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoaddiawr/gfhsaska.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=15872', 'data', 'file=/tmp/tmpoaddiawr/so7yjbj9.json', 'init=/tmp/tmpoaddiawr/gfhsaska.json', 'output', 'file=/tmp/tmpoaddiawr/prophet_model28txw9i0/prophet_model-20250912191800.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:18:00 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start proces

          ds           y
0 2025-03-12  216.450180
1 2025-03-13  209.167999
2 2025-03-14  212.968704
3 2025-03-17  213.477463
4 2025-03-18  212.170670


19:18:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoaddiawr/10nofybu.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoaddiawr/639qkvrm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=8525', 'data', 'file=/tmp/tmpoaddiawr/10nofybu.json', 'init=/tmp/tmpoaddiawr/639qkvrm.json', 'output', 'file=/tmp/tmpoaddiawr/prophet_modelv0u5dumc/prophet_model-20250912191800.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:18:00 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:18:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
Device set to use cpu



--- Prophet Predictions ---
2025-09-15: Predicted 236.88 (range 227.52 - 246.71)
2025-09-16: Predicted 237.19 (range 226.84 - 246.48)
2025-09-17: Predicted 238.84 (range 229.04 - 248.03)
2025-09-18: Predicted 238.66 (range 229.26 - 248.30)
2025-09-19: Predicted 238.71 (range 228.49 - 247.79)

--- LLM Summary ---
Here are Prophet's stock price predictions for AAPL: 2025-09-15: Predicted 236.88 (range 227.52 - 246.71) 2025-09-16: Predicted 237.19 (range 226.84 - 246.48) 2025-09-17: Predicted 238.84 (range 229.04 - 248.03) 2025-09-18: Predicted 238.66 (range 229.26 - 248.30) 2025-09-19: Predicted 238.71 (range 228.49 - 247.79)
